<a href="https://colab.research.google.com/github/ReemAhmed-as/DataAna2/blob/main/Online_retail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Change this path to where you have saved the Data folder in your drive
path = '/content/drive/My Drive/Colab Notebooks/online+retail'

# Check if the directory exists
if os.path.exists(path):
  os.chdir(path)
  print(f"Changed directory to {path}")
else:
  print(f"Directory not found: {path}")
  # You might want to create the directory if it doesn't exist:
  # os.makedirs(path)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mounted at /content/drive
Changed directory to /content/drive/My Drive/Colab Notebooks/online+retail


In [ ]:
# Import necessary libraries

from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules

# بس اول 300 الف
# Adjust the path to where the dataset is located
df = pd.read_excel('Online_Retail.xlsx',nrows=300000)

# View the first few rows of the dataset
print(df.head())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  


In [ ]:
# إزالة الصفوف المكررة
df = df.drop_duplicates()

print("\ndata sheap")
print(df.shape)

print("\nfirst 5 col")
print(df.head())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



data sheap
(297663, 8)

first 5 col
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  


In [ ]:
print("\n the number of the null value")
print(df.isnull().sum())

# إزالة الصفوف التي تحتوي على قيم مفقودة
df = df.dropna()
df.shape


 the number of the null value
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(213603, 8)

In [ ]:


# Data Cleaning
# Remove rows with missing CustomerID (they represent incomplete transactions)
df = df[df['CustomerID'].notna()]

# Remove canceled orders (Quantity < 0)
df = df[df['Quantity'] > 0]

# Create a basket (One-hot encoded format where each row is a transaction)
basket = (df.groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

# Convert the quantities into 1s and 0s (1 = bought, 0 = not bought)
def encode_units(x):
    return 1 if x >= 1 else 0

basket = basket.applymap(encode_units)



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-11-3257161cb068>:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(encode_units)


In [ ]:
# Perform FP-Growth to find frequent itemsets
# Set minimum support threshold (e.g., 0.02 means the itemset must appear in 2% of transactions)
frequent_itemsets = fpgrowth(basket, min_support=0.02, use_colnames=True)

# Display frequent itemsets
print("Frequent Itemsets:")
print(frequent_itemsets)

# Generate association rules based on frequent itemsets
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Display association rules
print("\nAssociation Rules:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Frequent Itemsets:
      support                                           itemsets
0    0.124730               (WHITE HANGING HEART T-LIGHT HOLDER)
1    0.075682                    (ASSORTED COLOUR BIRD ORNAMENT)
2    0.044453                         (HOME BUILDING BLOCK WORD)
3    0.036012                         (LOVE BUILDING BLOCK WORD)
4    0.032636                              (DOORMAT NEW ENGLAND)
..        ...                                                ...
242  0.024665  (LUNCH BAG RED RETROSPOT, LUNCH BAG SUKI DESIGN )
243  0.022414      (LUNCH BAG CARS BLUE, LUNCH BAG SUKI DESIGN )
244  0.020163  (LUNCH BAG SUKI DESIGN , LUNCH BAG PINK POLKADOT)
245  0.020913  (LUNCH BAG  BLACK SKULL., LUNCH BAG SUKI DESIGN )
246  0.020913                    (PARTY BUNTING, SPOTTY BUNTING)

[247 rows x 2 columns]

Association Rules:
                             antecedents  \
0           (ALARM CLOCK BAKELIKE GREEN)   
1            (ALARM CLOCK BAKELIKE RED )   
2    (WOODEN PICTURE FRAM